# Cvičení 1 - Úvod do neuronových sítí

V tomto cvičení si ukážeme prácis s jazykem **Python** a jeho knihovnami, především [PyTorch](https://pytorch.org/).

Obdobným způsobem se pracuje i s dalšími používanými knihovnami jako např. [TensorFlow](https://www.tensorflow.org/), [JAX](https://docs.jax.dev/en/latest/), [Flux.jl](https://fluxml.ai/) nebo [Deep Learning Toolbox](https://www.mathworks.com/products/deep-learning.html).

Pro detailnější tutoriál k PyTorch jsou dostupné i [oficiální tutoriály](https://pytorch.org/tutorials/).

## Instalace v prostředí uv a Jupyter

Nainstalujeme Jupyter dle pokynů z [webu](https://jupyter.org/install). Obdobně s balíčkovacím systémem [uv](https://docs.astral.sh/uv/).

Vytvoříme uv balíček a propojíme ho s Jupyterem
```bash
$ uv init
$ uv add --dev ipykernel
$ uv run ipython kernel install --user --env VIRTUAL_ENV $(pwd)/.venv --name=tzn
```

Pro dostupnost tohoto kernelu musíme restartovat Jupyter.

Nainstalujeme potřebné balíčky

```bash
$ uv add matplotlib numpy scikit-learn torch torch-geometric torchvision
$ uv add torch-cluster -f https://data.pyg.org/whl/torch-2.8.0+cpu.html
```

## Úkol 1

Zprovozněte své prostředí tak, aby se následující buňka spustila bez chyb a zobrazila něco jako `Torch version: 2.8.0`.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

print("Torch version:", torch.__version__)

## Tenzory a práce s nimi v PyTorch

Nejprve se podívejme na to, co se vlastně při standardním využití děje na pozadí - manipulace s tenzory a automatické počítání gradientu.

Tenzor je vlastně název pro $n$ rozměrné pole - tj. jako `ndarray` v numpy.

Převod numpy nebo python matice na konstantní tenzor:

In [ ]:
a = torch.tensor([[1, 2], [3,4]])
print(a)

Převod zpět do numpy:

In [ ]:
print(a.numpy())

Tensor vždy obsahuje základní atributy `shape` a `dtype`:

In [ ]:
print('Rozměr:', a.shape)
print('Datový typ:', a.dtype)

Další způsoby, jak vytvořit Tensor:

In [ ]:
print(torch.ones((1, 3)))
print(torch.zeros((1, 3)))
print(torch.randn((1, 4))) # Normální rozdělení
print(torch.rand((1, 4))) # Uniformní rozdělení na [0, 1)

S tenzory můžeme dělat běžné operace - maticové atd. Vyrobíme si nějaké tenzory (musí být stejného datového typu):

In [ ]:
a = torch.tensor([[1, 2], [3,4]], dtype = torch.float32)
b = torch.ones((2, 1))
c = torch.randn((2, 1))
print(a)
print(b)
print(c)

Maticový součin:

In [ ]:
print(a.matmul(b))

Součet po složkách:

In [ ]:
print(b.add(c))

Kvadrát po složkách:

In [ ]:
print(a.square())

Eukleidovská norma vektoru:

In [ ]:
print(b.norm())

Hodnost matice:

In [ ]:
print(torch.linalg.matrix_rank(a))

Aplikace funkce sinus po složkách:

In [ ]:
print(torch.sin(a))

Boolovský idikátor toho, zda je daná složka matice větší než 1:

In [ ]:
print(a.greater(1))

## Úkol 2 - základní práce s daty

Načteme dataset [MNIST](https://en.wikipedia.org/wiki/MNIST_database) a zkusíme spočítat průměrné hodnoty jeho jednotlivých feature. První spuštění následující buňky může trvat déle, než se dataset stáhne.

In [ ]:
data = datasets.MNIST(
    root = "data",
    download = True,
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0, 1)])
)

Načetli jsme obrázky z datasetu MNIST. Nyní zkuste spočítat průměrnou hodnotu každé feature napříč všemi obrázky. Využijte třídu [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).

## Automatické počítání gradientu

Objekt, který se používá na práci s proměnlivým obsahem, je standardní `torch.Tensor`. Nejzajímavější na něm je **automatické počítání gradientu** vzhledem k operacím prováděným s ním.

K tomu se používá parametr `requires_grad = True`, pomocí kterého se zaznamenávají operace se sledovaným tenzorem.
Gradient pak spočteme voláním metody `fun.backward()`, který spočte derivaci `fun` podle každé složky `x`. Pokud je `fun` tensor rozměru většího než 1, je potřeba funkci `backward` předat tensor, vzhledem ke kterému se má Jakobián spočítat.

In [ ]:
x = torch.tensor([1, 2, 3], dtype = torch.float32, requires_grad = True)
print(x)

In [ ]:
f = torch.square(x)
f.backward(torch.ones_like(f))

print('Derivace x^2:', x.grad)
with torch.no_grad():
    print('2*x:', 2*x)

In [ ]:
x = torch.tensor([1, 2, 3], dtype = torch.float32, requires_grad = True)
print(x)

In [ ]:
f = torch.sin(x)
f.backward(torch.ones_like(f))
    
print('Derivace sin(x):', x.grad)
with torch.no_grad():
    print('cos(x):', torch.cos(x))

## Jednoduchá síť na MNIST

Definujeme základní parametry modelu:

In [ ]:
batch_size = 50
hidden_layer_width = 100
output_width = 10
learning_rate = 0.01

Připravíme si data do tříd DataLoader:

In [ ]:
data_train = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0, 1)])
)
data_test = datasets.MNIST(
    root = "data",
    train = False,
    download = True,
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0, 1)])
)

dataloader_train = DataLoader(data_train, batch_size = batch_size, shuffle = True)
dataloader_test = DataLoader(data_test, batch_size = batch_size, shuffle = True)

Vytvoříme jednoduchý model neuronové sítě pomocí `torch.nn.Sequential` což je třída, která za sebe poskládá vrstvy, které do ní vložíme (viz předchozí cvičení).

Vstupní vrstva je typu `torch.nn.Flatten`, tedy vrstva která převede obrázky do tvaru vektoru (opět automatizace z minulého cvičení).

Výslkedkem je instance [tf.nn.Module](https://pytorch.org/docs/main/generated/torch.nn.Module.html).

In [ ]:
model = nn.Sequential()
model.append(nn.Flatten())

Přidáme jednu skrytou vrstvu se 100 neurony a aktivační funkcí `tanh`

In [ ]:
model.append(nn.Linear(data_train.data.shape[1] * data_train.data.shape[2], hidden_layer_width))
model.append(nn.Tanh())

Přidáme výstupní vrstvu s 10 neurony (= počet tříd) a aktivační funkcí softmax
$$ \sigma \left( \vec{z} \right)_i = \frac{e^{\vec{z}_i}}{\sum_{j = 1}^K e^{\vec{z}_j}} $$

In [ ]:
model.append(nn.Linear(hidden_layer_width, output_width))
model.append(nn.Softmax(dim=1))

Vytisknutí modelu nám vytiskne kompletní informace:

In [ ]:
model

Takto vytvořený model zároveň funguje jako funkce a můžeme ho rovnou aplikovat na data.

Výskedkem budou vektory pravděpodobností. Protože jsme ale model zatím netrénovali, výsledky budou prakticky náhodné

In [ ]:
model(data_train[0][0])

Výslednou "předpověď" modelu pak bereme jako položku s nejvyšší pravděpodobností:

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(24):
    plt.subplot(4, 6, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data_train[i][0].reshape(28, 28), cmap = plt.cm.bone)
    plt.title(model(data_train[i][0]).detach().argmax(axis = 1).item())

## Trénování modelu

Nyní můžeme model natrénovat. Co budeme dále potřebovat:
* loss - ztrátová funkce, kterou chceme při trénování minimalizovat,
* optimizer - funkce, která pracuje s gradientem a udělá krok gradientního sestupu,

Poznámky:
* `torch.nn.NLLLoss` je cross-entropie, kde vstup je pravděpodobnostní rozdělení
* `torch.nn.CrossEntropyLoss()` je cross-entropie, kde vstup jsou váhy jednotlivých tříd (tj. jejich suma nemusí být 1)

Jinými slovy `CrossEntropyLoss` kombinuje `NLLLoss` a `Softmax`.

Použijeme stochastický gradientní sestup.

In [ ]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 10

K trénování modelu použijeme `DataLoader`, obdobně jako v minulém cvičení. 
Trénování v mini batchích znamená, že se dataset rozdělí na bločky (mini batche) a pak se pro každý bloček spočítá ztrátová funkce a udělá jede krok gradientního sestupu. Jedna epocha pak znamená projití celého datasetu. V jedné epoše tedy dojde k mnoha krokům gradientního sestupu.

In [ ]:
def calculate_accuracy(model, dataloader):
    num_correct = 0
    
    with torch.no_grad():
        for (X, y) in dataloader:
            pred = model(X)
            num_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    accuracy = num_correct / len(dataloader.dataset)
    return accuracy

In [ ]:
def calculate_loss_accuracy(model, dataloader, loss_fn):
    loss = 0
    num_correct = 0
    
    with torch.no_grad():
        for (X, y) in dataloader:
            pred = model(X)
            loss += loss_fn(pred, y).item()
            num_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    loss /= len(dataloader)
    accuracy = num_correct / len(dataloader.dataset)
    return loss, accuracy

In [ ]:
def train_model(model, loss_fn, optimizer, epochs, dataloader_train, dataloader_test, early_stopper = None, log_period = 10000):
    for epoch in range(epochs):
        processed_since_log = 0
        for batch, (X, y) in enumerate(dataloader_train):
            model.train()
            pred = model(X)
            loss = loss_fn(pred, y)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            processed_since_log += dataloader_train.batch_size

            if processed_since_log >= log_period:
                current = min((batch + 1) * dataloader_train.batch_size, len(data_train))
                loss = loss.item()
                model.eval()
                train_acc = calculate_accuracy(model, dataloader_train)
                test_loss, test_acc = calculate_loss_accuracy(model, dataloader_test, loss_fn)
                print(f"train loss: {loss:>7f}  test loss: {test_loss:>7f}  train accuracy: {train_acc:>3f}  test accuracy: {test_acc:>3f}  [sample {current:>5d}/{len(data_train):>5d}] [epoch {epoch+1:>2d}/{epochs:>2d}]")
                processed_since_log -= log_period

In [ ]:
train_model(model, loss_fn, optimizer, epochs, dataloader_train, dataloader_test)

### Evaluace modelu a predikce

K evaluaci modelu na testovací množině využijeme výše implementované funkce.

In [ ]:
model.eval()
test_loss, test_acc = calculate_loss_accuracy(model, dataloader_test, loss_fn)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Pokud chceme přímo "syrové" hodnoty, můžeme použít model jako funkci:

In [ ]:
predictions = model(data_test[0][0])
print('Predictions shape:', predictions.shape)
print('Predikce pravděpodobností pro první obrázek:', predictions[0, 0].item())

Pro získání predikcí tříd můžeme použít `torch.argmax`, která vrátí index maximální pravděpodobnosti.

In [ ]:
Y_pred = predictions.detach().argmax(dim = 1)
print('Predikce pravděpodobností pro první obrázek:', predictions.detach())
print('Predikce labelu pro první obrázek:', Y_pred[0].item())

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(24):
    plt.subplot(4, 6, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data_test[i][0].reshape(28, 28), cmap = plt.cm.bone)
    plt.title(model(data_test[i][0]).detach().argmax(axis = 1).item())

## Úkol 3 - Manuální vytvoření modelu

Definujeme základní parametry modelu:

In [ ]:
batch_size = 50
hidden_layer_width = 100
output_width = 10
learning_rate = 0.01
epochs = 10
loss_fn = nn.CrossEntropyLoss()

Definuji si proměnné pro neuronovou síť s 1 skrytou vrstvou velikosti 100:

In [ ]:
# Průměr 0, směrodatná odchylka 0.1
W1 = (0.1 * torch.randn((data_train[0][0].shape[1] * data_train[0][0].shape[2], hidden_layer_width))).clone().requires_grad_(True)
b1 = torch.zeros((hidden_layer_width,), requires_grad = True)
W2 = (0.1 * torch.randn((hidden_layer_width, output_width))).clone().requires_grad_(True)
b2 = torch.zeros((output_width,), requires_grad = True)

Definujeme funkci vykonávající dopředný chod sítě:

In [ ]:
def predict(inputs):
    # TODO

Natrénujeme model:

In [ ]:
log_period = 10000

for epoch in range(epochs):
    processed_since_log = 0
    for batch, (X, y) in enumerate(dataloader_train):
        pred = predict(X)
        loss = loss_fn(pred, y)
        loss.backward()

        for variable in [W1, b1, W2, b2]:
            with torch.no_grad():
                variable -= variable.grad * learning_rate
            variable.grad = None

        processed_since_log += dataloader_train.batch_size

        if processed_since_log >= log_period:
            current = min((batch + 1) * dataloader_train.batch_size, len(data_train))
            loss = loss.item()
            train_acc = calculate_accuracy(predict, dataloader_train)
            test_loss, test_acc = calculate_loss_accuracy(predict, dataloader_test, loss_fn)
            print(f"train loss: {loss:>7f}  test loss: {test_loss:>7f}  train accuracy: {train_acc:>3f}  test accuracy: {test_acc:>3f}  [sample {current:>5d}/{len(data_train):>5d}] [epoch {epoch+1:>2d}/{epochs:>2d}]")
            processed_since_log -= log_period

## Parametry trénování

V předchozím příkladu jsme nejrůznější hyper-parametery modelu "stříleli od boku", pojďme se tedy podívat, jaké mohou mít hodnoty.

### Architektura sítě

- Počet vrstev
- Šířka vrstev
- Aktivační funkce
    - linear
    - tanh
    - sigmoid
    - hard sigmoid
    - relu
    - selu
    - softmax

In [ ]:
plt.figure(figsize=(20, 20))
i = 1
for activationFunction in [nn.Tanh(), nn.Hardtanh(), nn.Sigmoid(), nn.Hardsigmoid(), nn.ReLU(), nn.LeakyReLU(), nn.SELU(), nn.ELU()]:
    plt.subplot(4, 4, i)
    i += 1
    plt.grid(True)
    xs = torch.linspace(-5, 5, 100);
    ys = activationFunction(xs)
    plt.plot(xs, ys)
    plt.title(type(activationFunction).__name__)

## Ztrátová funkce

- Mean square error
    $$ \operatorname{MSE}=\frac{1}{n}\sum_{i=1}^n(Y_i-\hat{Y_i})^2. $$
- Hinge
    $$ \operatorname{Hinge} = \sum_{i=1}^K \max(0, 1- Y_i \cdot \hat{Y_i}) $$
- KL divergence
    $$ \operatorname{KL} = - \sum_{i=1}^K \hat{Y_i} \cdot \log(\frac{Y_i}{\hat{Y_i}}) $$
- Cross-entropy
    $$ \operatorname{crossentropy} = - \sum_{i=1}^K Y_i \cdot \log(\hat{Y_i}) $$

## Optimalizační algoritmus

- SGD
- RMSProp
- Adagrad
- Adam
- Adadelta
- Adamax
- Nadam

### Learning schedule

Volitelně můžeme optimalizační algoritmy doplnit o tzv. weight decay - tj. postupnou změnu learning rate. Například

In [ ]:
initial_learning_rate = 0.01
batch_count = 64
optimizer = torch.optim.SGD(model.parameters(), weight_decay=0.001)

Pro pokročilejší algoritmy typicky není weight decay potřeba, protože mají podobnou funkčnost již zabudovanou.

## Postupné používání datasetu

To, jak je dataset použit při trénování je určeno 2 parametry

- Epochy: kolikrát je celý dataset použit
- Batching: Dataset není použit prvek po prvku, ale vždy je najednou trénováno pomocí celé batch (mini-batch)

# Úkol 4

Zkuste upravit model na MNISTu a získat co nejlepší přesnost na testovací sadě pomocí výše popsaných parametrů